In [1]:
import numpy as np
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from src.score import *
import re

import generate_data as gd

In [2]:
class PeriodicPadding2D(tf.keras.layers.Layer):
    def __init__(self, pad_width, **kwargs):
        super().__init__(**kwargs)
        self.pad_width = pad_width

    def call(self, inputs, **kwargs):
        if self.pad_width == 0:
            return inputs
        inputs_padded = tf.concat(
            [inputs[:, :, -self.pad_width:, :], inputs, inputs[:, :, :self.pad_width, :]], axis=2)
        # Zero padding in the lat direction
        inputs_padded = tf.pad(inputs_padded, [[0, 0], [self.pad_width, self.pad_width], [0, 0], [0, 0]])
        return inputs_padded

    def get_config(self):
        config = super().get_config()
        config.update({'pad_width': self.pad_width})
        return config


class PeriodicConv2D(tf.keras.layers.Layer):
    def __init__(self, filters,
                 kernel_size,
                 conv_kwargs={},
                 **kwargs, ):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.conv_kwargs = conv_kwargs
        if type(kernel_size) is not int:
            assert kernel_size[0] == kernel_size[1], 'PeriodicConv2D only works for square kernels'
            kernel_size = kernel_size[0]
        pad_width = (kernel_size - 1) // 2
        self.padding = PeriodicPadding2D(pad_width)
        self.conv = Conv2D(
            filters, kernel_size, padding='valid', **conv_kwargs
        )

    def call(self, inputs):
        return self.conv(self.padding(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({'filters': self.filters, 'kernel_size': self.kernel_size, 'conv_kwargs': self.conv_kwargs})
        return config

In [18]:
def build_cnn(filters, kernels, input_shape, dr=0):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    for f, k in zip(filters[:-1], kernels[:-1]):
        x = PeriodicConv2D(f, k)(x)
        x = Activation(activation='sigmoid')(x)
    output = PeriodicConv2D(filters[-1], kernels[-1])(x)
    return keras.models.Model(input, output)

def build_cnn2(filters, kernels, input_shape, dr=0):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    for f, k in zip(filters[:-1], kernels[:-1]):
        x = PeriodicConv2D(f, k)(x)
        x = LeakyReLU()(x)
    output = PeriodicConv2D(filters[-1], kernels[-1])(x)
    return keras.models.Model(input, output)

def create_predictions(model, dg):
    """Create non-iterative predictions"""
    preds_un = xr.DataArray(
        model.predict(dg),
        dims=['time', 'lat', 'lon', 'level'],
        coords={'time': dg.valid_time, 'lat': dg.data.lat, 'lon': dg.data.lon,
                'level': dg.data.isel(level=dg.output_idxs).level,
                'level_names': dg.data.isel(level=dg.output_idxs).level_names
               },
    )
    # Unnormalize
    preds = (preds_un * (dg.max_data.isel(level=dg.output_idxs).values - dg.min_data.isel(level=dg.output_idxs).values) +
             dg.min_data.isel(level=dg.output_idxs).values)
    unique_vars = list(set([l.split('_')[0] for l in preds.level_names.values])); unique_vars

    das = []
    for v in unique_vars:
        idxs = [i for i, vv in enumerate(preds.level_names.values) if vv.split('_')[0] in v]
        #print(v, idxs)
        da = preds.isel(level=idxs).squeeze().drop('level_names')
        if not 'level' in da.dims: da.drop('level')
        das.append({v: da})
    return preds_un, xr.merge(das, compat = 'override').drop('level')

In [20]:
def predictions(var_name, valid):
    dg_train, dg_valid, dg_test = gd.create_data_max_min(var_name)

    if var_name == 'specific_humidity':
        cnn = build_cnn2([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 4))
    elif var_name == 'pot_vort':
        cnn = build_cnn2([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 4))
    elif var_name == 'const':
        cnn = build_cnn2([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 5))
    elif var_name == 'orig':
        cnn = build_cnn2([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 2))
    else:
        cnn = build_cnn2([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 3))

    cnn.compile(keras.optimizers.Adam(1e-4), 'mse')

    cnn.load_weights('/rds/general/user/mc4117/home/WeatherBench/saved_models/72_gpu_annual_maxmin' + str(var_name) + '.h5')

    number_of_forecasts = 1

    pred_ensemble=np.ndarray(shape=(2, 17448, 32, 64, number_of_forecasts),dtype=np.float32)
    print(pred_ensemble.shape)
    forecast_counter=np.zeros(number_of_forecasts,dtype=int)

    for j in range(number_of_forecasts):
        print(j)
        preds, output = create_predictions(cnn, dg_test)
        pred2 = np.asarray(output.to_array(), dtype=np.float32).squeeze()
        pred_ensemble[:,:,:,:,j]=pred2
        forecast_counter[j]=j+1
        filename2 = '/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annual' + str(var_name)
        np.save(filename2 + '.npy', pred_ensemble)
        


    return preds, compute_weighted_rmse(output, valid).compute()

In [16]:
DATADIR = '/rds/general/user/mc4117/home/WeatherBench/data/'

z500_valid = load_test_data(f'{DATADIR}geopotential_500', 'z')
t850_valid = load_test_data(f'{DATADIR}temperature_850', 't')
valid = xr.merge([z500_valid, t850_valid])


In [19]:
preds_orig, rmse = predictions('orig', valid)
rmse

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.366
    z        float64 729.5

In [21]:
preds_sh, rmse_sh = predictions('specific_humidity', valid)
rmse_sh

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.334
    z        float64 726.7

In [22]:
preds_pv, rmse_pv = predictions('pot_vort', valid)
rmse_pv

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.364
    z        float64 729.2

In [24]:
preds_const, rmse_const = predictions('const', valid)
rmse_const

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.288
    z        float64 730.8

In [25]:
preds_2mt, rmse_2mt = predictions('2m temp', valid)
rmse_2mt

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.344
    z        float64 728.7

In [23]:
preds_sr, rmse_sr = predictions('solar rad', valid)
rmse_sr

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.371
    z        float64 726.6

In [ ]:
#output_orig = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualorig.npy')
#output_sh = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualspecific_humidity.npy')
#output_pv = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualpot_vort.npy')
#output_const = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualconst.npy')
#output_2mt = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annual2m temp.npy')
#output_sr = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualsolar rad.npy')

In [ ]:
#output_mean = sum([output_sh + output_pv + output_const + output_2mt + output_sr])/5

#dg_train, dg_valid, dg_test = gd.create_data('orig')

#output_mean_ds = xr.Dataset({
#    'z': xr.DataArray(
#        output_mean[0, ..., 0],
#        dims=['time', 'lat', 'lon'],
#        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
#                },
#    ),
#    't': xr.DataArray(
#        output_mean[1, ..., 0],
#        dims=['time', 'lat', 'lon'],
#        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
#                },
#    )
#})


In [ ]:
#compute_weighted_rmse(output_mean_ds, valid).compute()

In [ ]:
stack_out = np.stack([preds_orig, preds_sh, preds_pv, preds_const, 
                      preds_2mt,  preds_sr])  

In [ ]:
dg_train, dg_valid, dg_test = gd.create_data('orig')
X1, y1 = dg_test[0]

for i in range(1, len(dg_test)):
    X2, y2 = dg_test[i]
    y1 = np.concatenate((y1, y2))

In [ ]:
stack_out.shape

In [ ]:
stack_test_list = []

for i in range(6):
    stack_test_list.append(stack_out[i])

In [ ]:
print(stack_test_list[0].shape)
print(y1.shape)

In [ ]:
from tensorflow.keras.layers import concatenate

def my_init(shape, dtype=None):
    print(shape)
    return tf.ones(shape, dtype=dtype)/6

def build_stack_model(input_shape, stack_list):
    # concatenate merge output from each model
    input_list = [Input(shape=input_shape) for i in range(len(stack_list))]
    merge = concatenate(input_list)
    hidden = Dense(6, activation='relu', kernel_initializer = my_init, bias_initializer = tf.zeros)(merge)
    hidden = Dense(64, activation='relu')(hidden)
    output = Dense(2)(hidden)
    return keras.models.Model(input_list, output)

In [ ]:
ensemble_model = build_stack_model((32, 64, 2), stack_test_list)

ensemble_model.compile(keras.optimizers.Adam(1e-4), 'mse')

def scheduler(epoch, lr):
    if epoch< 5:
        return lr #/(2*epoch+1)
    elif epoch < 10:
        return lr/10
    elif epoch < 15:
        return lr/20
    else:
        return lr/100

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)   

ensemble_model.fit(x = stack_test_list, y = y1, epochs = 200, validation_split = 0.2, shuffle = True
                  , callbacks = [tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        min_delta=0,
                        patience=2,
                        verbose=1, 
                        mode='auto'
                    )]) #, callbacks = [callback])

In [ ]:
    fc = ensemble_model.predict(stack_test_list)
    preds_un = xr.DataArray(
        fc,
        dims=['time', 'lat', 'lon', 'level'],
        coords={'time': dg_test.valid_time, 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                'level': dg_test.data.isel(level=dg_test.output_idxs).level,
                'level_names': dg_test.data.isel(level=dg_test.output_idxs).level_names
               },
    )
    # Unnormalize
    preds = (preds_un * dg_test.std.isel(level=dg_test.output_idxs).values +
             dg_test.mean.isel(level=dg_test.output_idxs).values)
    unique_vars = list(set([l.split('_')[0] for l in preds.level_names.values])); unique_vars

    das = []
    for v in unique_vars:
        idxs = [i for i, vv in enumerate(preds.level_names.values) if vv.split('_')[0] in v]
        #print(v, idxs)
        da = preds.isel(level=idxs).squeeze().drop('level_names')
        if not 'level' in da.dims: da.drop('level')
        das.append({v: da})
    fc_unnorm = xr.merge(das, compat = 'override').drop('level')

In [ ]:

real_unnorm =y1* dg_test.std.isel(level=[0,1]).values+dg_test.mean.isel(level=[0,1]).values

real_ds = xr.Dataset({
    'z': xr.DataArray(
        real_unnorm[..., 0],
        dims=['time', 'lat', 'lon'],
        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                },
    ),
    't': xr.DataArray(
        real_unnorm[..., 1],
        dims=['time', 'lat', 'lon'],
        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                },
    )
})

In [ ]:
compute_weighted_rmse(fc_unnorm, real_ds).compute()

In [ ]:
stack_total = sum(stack_test_list)/len(stack_test_list)

In [ ]:
    preds_total_un = xr.DataArray(
        stack_total,
        dims=['time', 'lat', 'lon', 'level'],
        coords={'time': dg_test.valid_time, 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                'level': dg_test.data.isel(level=dg_test.output_idxs).level,
                'level_names': dg_test.data.isel(level=dg_test.output_idxs).level_names
               },
    )
    # Unnormalize
    preds_total = (preds_total_un * dg_test.std.isel(level=dg_test.output_idxs).values +
             dg_test.mean.isel(level=dg_test.output_idxs).values)
    unique_vars = list(set([l.split('_')[0] for l in preds_total.level_names.values])); unique_vars

    das = []
    for v in unique_vars:
        idxs = [i for i, vv in enumerate(preds_total.level_names.values) if vv.split('_')[0] in v]
        #print(v, idxs)
        da = preds_total.isel(level=idxs).squeeze().drop('level_names')
        if not 'level' in da.dims: da.drop('level')
        das.append({v: da})
    fc_unnorm_total = xr.merge(das, compat = 'override').drop('level')

In [ ]:
compute_weighted_rmse(fc_unnorm_total, real_ds).compute()